# Описание
В данном ноутбуке выполняется преобразование текстов из столбца 'source_name_model' в усреднённые эмбеддинги токенов с использованием предобученной модели 'cointegrated/rubert-tiny2'. Для каждого текста рассчитывается mean pooling по скрытым представлениям всех непаддинговых токенов. Полученные эмбеддинги, вместе с дополнительными признаками ('cat_id', 'самовывоз', 'возможность доставки', 'гарантия'), используются для формирования финального датасета, пригодного для последующего обучения модели машинного обучения.

# Import

In [ ]:
import pandas as pd
import numpy as np

import torch
from transformers import AutoTokenizer, AutoModel

from tqdm.auto import tqdm
tqdm.pandas(desc='Tokenizing rows')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
df = pd.read_parquet('full_df.parquet')

In [ ]:
model_name = 'cointegrated/rubert-tiny2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

# future_extraction

In [ ]:
def cleaned_text(text) -> str:
    """
    Простая очистка текста.
    """
    text = str(
        text
    ) if text is not None else ''  # преобразуем text в строку, если это не строка
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s.,*!?:-]', '',
                  text)  # удаление лишних символов (кроме пунктуации)
    text = re.sub(r'\s+', ' ', text).strip()  # удаление лишних пробелов
    return text


# применяем ко всему датасету
df['source_name_model'] = df['source_name_model'].apply(cleaned_text)

In [ ]:
def get_bert_embedding(text: str) -> np.ndarray:
    """
    Получает эмбеддинг текста с использованием mean pooling 
    по всем токенам (кроме паддинга).
    
    :param text: входной текст
    :return: усреднённый эмбеддинг по всем токенам
    """
    tokens = tokenizer(text,
                       return_tensors='pt',
                       truncation=True,
                       padding='max_length',
                       max_length=27)

    # переносим данные на GPU, если он есть
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():  # выключаем градиенты
        output = model(**tokens)
        last_hidden_state = output.last_hidden_state  # [batch_size, seq_len, hidden_dim]
        attention_mask = tokens['attention_mask']  # [batch_size, seq_len]

        # применяем attention mask: обнуляем эмбеддинги паддингов
        mask = attention_mask.unsqueeze(-1).expand(last_hidden_state.size())
        masked_embeddings = last_hidden_state * mask

        # усреднение по непаддинговым токенам
        summed = masked_embeddings.sum(dim=1)
        counts = mask.sum(dim=1)  # число непаддинговых токенов
        mean_pooled = summed / counts

    return mean_pooled.cpu().numpy().squeeze()

In [ ]:
# извлечение значений из датасета
texts = df['source_name_model'].tolist()
labels = df['cat_id'].values  # метки классов
pickup = df['самовывоз'].values
delivery = df['возможность доставки'].values
guarantee = df['гарантия'].values

In [ ]:
# преобразуем тексты в эмбеддинги
embeddings = np.array([get_bert_embedding(text) for text in tqdm(texts)])

In [ ]:
# состовляем финальный датасет для последующего обучения на нем модели
final_dataset = pd.DataFrame(embeddings)
final_dataset['pickup'] = pickup
final_dataset['delivery'] = delivery
final_dataset['guarantee'] = guarantee
final_dataset['labels'] = labels

In [ ]:
# сохраняем финальный датасет
final_dataset.to_parquet('final_dataset.parquet', index=False)